In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/coco-instancesegmentationmask_rcnn_r_101_fpn_3x/pytorch/default/1/model_final_a3ec72.pkl
/kaggle/input/read-icdar-2019/read-icdar2019/train.json
/kaggle/input/read-icdar-2019/read-icdar2019/test.json
/kaggle/input/read-icdar-2019/read-icdar2019/eval.json
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-1280.jpg
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-1672.jpg
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-1428.jpg
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-1332.jpg
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-0633.jpg
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-1100.jpg
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-1728.jpg
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-2125.jpg
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-1993.jpg
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-1372.jpg
/kaggle/input/read-icdar-2019/read-icdar2019/eval/cPAS-1536.jpg
/kaggle/input/read-icdar-20

In [2]:
!pip install layoutparser torchvision && pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"
!pip install 'git+https://github.com/facebookresearch/detectron2.git@ff53992b1985b63bd3262b5a36167098e3dada02'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 83.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.8 MB/s eta 0:00:000:

In [3]:
# ==================== DATA PREPARATION ====================
def load_simple_coco_json(json_path):
    """Load COCO format JSON file"""
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data
    

In [4]:
from detectron2.data.datasets import load_coco_json  # must come after your own definition or use alias

In [5]:
# PAGE-XML Detectron2 Mask R-CNN Training
# Fine-tuning Mask R-CNN with ResNet-101 + FPN on PAGE-XML derived COCO dataset
# Classes: textline (0), baseline (1)

# Install required packages
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu118/torch2.0/index.html
!pip install pycocotools

In [6]:
# Import necessary libraries
import os
import json
import shutil
import random
import cv2
import numpy as np
from collections import defaultdict
import torch
import matplotlib.pyplot as plt

# Detectron2 imports
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, default_setup
from detectron2.model_zoo import model_zoo
from detectron2.data.datasets import load_coco_json
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.evaluation import COCOEvaluator
from detectron2.utils.logger import setup_logger

# Setup logger
setup_logger()

<_Logger detectron2 (DEBUG)>

In [7]:
# ==================== DATA PREPARATION ====================

def load_simple_coco_json(json_path):
    """Load COCO format JSON file"""
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

In [8]:
def compute_area_from_segmentation(segmentation):
    """Compute area from segmentation polygon or RLE"""
    if isinstance(segmentation, list) and len(segmentation) > 0:
        # Polygon format
        from pycocotools import mask as maskUtils
        if isinstance(segmentation[0], list):
            # Multiple polygons
            rles = maskUtils.frPyObjects(segmentation, 1000, 1000)  # dummy height/width
            rle = maskUtils.merge(rles)
        else:
            # Single polygon
            rle = maskUtils.frPyObjects([segmentation], 1000, 1000)[0]
        return maskUtils.area(rle)
    elif isinstance(segmentation, dict):
        # RLE format
        from pycocotools import mask as maskUtils
        return maskUtils.area(segmentation)
    else:
        # Fallback: compute from bbox
        return 0

In [9]:
def compute_area_from_bbox(bbox):
    """Compute area from bounding box [x, y, width, height]"""
    if len(bbox) >= 4:
        return bbox[2] * bbox[3]  # width * height
    return 0

In [10]:
def fix_annotations(annotations):
    """Fix annotations by adding missing area field"""
    fixed_annotations = []
    
    for ann in annotations:
        # Create a copy of the annotation
        fixed_ann = ann.copy()
        
        # Add area field if missing
        if 'area' not in fixed_ann or fixed_ann['area'] == 0:
            area = 0
            
            # Try to compute from segmentation first
            if 'segmentation' in fixed_ann and fixed_ann['segmentation']:
                try:
                    area = compute_area_from_segmentation(fixed_ann['segmentation'])
                except:
                    area = 0
            
            # Fallback to bbox if area is still 0
            if area == 0 and 'bbox' in fixed_ann:
                area = compute_area_from_bbox(fixed_ann['bbox'])
            
            # Final fallback - use a small default area
            if area == 0:
                area = 1.0
                
            fixed_ann['area'] = float(area)
        
        # Ensure iscrowd field exists
        if 'iscrowd' not in fixed_ann:
            fixed_ann['iscrowd'] = 0
            
        fixed_annotations.append(fixed_ann)
    
    return fixed_annotations

In [11]:
def merge_datasets(train_json_path, eval_json_path):
    """Merge train and eval COCO datasets"""
    print("Loading datasets...")
    train_data = load_simple_coco_json(train_json_path)
    eval_data = load_simple_coco_json(eval_json_path)
    
    # Merge images - need to reassign IDs to avoid conflicts
    merged_images = []
    image_id_mapping = {}
    current_image_id = 1
    
    # Process train images
    for img in train_data['images']:
        old_id = img['id']
        img['id'] = current_image_id
        # Add source info for later copying
        img['source'] = 'train'
        image_id_mapping[old_id] = current_image_id
        merged_images.append(img)
        current_image_id += 1
    
    # Process eval images
    eval_image_id_offset = {}
    for img in eval_data['images']:
        old_id = img['id']
        img['id'] = current_image_id
        img['source'] = 'eval'
        eval_image_id_offset[old_id] = current_image_id
        merged_images.append(img)
        current_image_id += 1
    
    # Merge annotations - update image_ids and reassign annotation IDs
    merged_annotations = []
    current_ann_id = 1
    
    # Process train annotations
    for ann in train_data['annotations']:
        ann['id'] = current_ann_id
        ann['image_id'] = image_id_mapping[ann['image_id']]
        merged_annotations.append(ann)
        current_ann_id += 1
    
    # Process eval annotations
    for ann in eval_data['annotations']:
        ann['id'] = current_ann_id
        ann['image_id'] = eval_image_id_offset[ann['image_id']]
        merged_annotations.append(ann)
        current_ann_id += 1
    
    # Fix annotations - add missing area fields
    print("Fixing annotations (adding missing area fields)...")
    merged_annotations = fix_annotations(merged_annotations)
    
    # Use categories from train data (should be same as eval)
    merged_data = {
        'images': merged_images,
        'annotations': merged_annotations,
        'categories': train_data['categories']
    }
    
    return merged_data

In [12]:
def split_dataset(merged_data, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    """Split merged dataset into train/val/test"""
    random.seed(seed)
    
    images = merged_data['images']
    annotations = merged_data['annotations']
    categories = merged_data['categories']
    
    # Create image_id to annotations mapping
    image_to_anns = defaultdict(list)
    for ann in annotations:
        image_to_anns[ann['image_id']].append(ann)
    
    # Shuffle images
    random.shuffle(images)
    
    # Calculate split indices
    total_images = len(images)
    train_end = int(total_images * train_ratio)
    val_end = train_end + int(total_images * val_ratio)
    
    # Split images
    train_images = images[:train_end]
    val_images = images[train_end:val_end]
    test_images = images[val_end:]
    
    def create_split_data(split_images):
        split_image_ids = set(img['id'] for img in split_images)
        split_annotations = []
        for img_id in split_image_ids:
            split_annotations.extend(image_to_anns[img_id])
        
        return {
            'images': split_images,
            'annotations': split_annotations,
            'categories': categories
        }
    
    train_data = create_split_data(train_images)
    val_data = create_split_data(val_images)
    test_data = create_split_data(test_images)
    
    return train_data, val_data, test_data

In [13]:
def copy_images(image_list, output_dir):
    """Copy images from source directories to destination directory"""
    os.makedirs(output_dir, exist_ok=True)
    
    for img in image_list:
        filename = img['file_name']
        source = img['source']
        
        # Source path based on original location
        if source == 'train':
            src_path = f'/kaggle/input/read-icdar-2019/read-icdar2019/train/{filename}'
        else:  # eval
            src_path = f'/kaggle/input/read-icdar-2019/read-icdar2019/eval/{filename}'
        
        if not os.path.exists(src_path):
            print(f"Warning: Could not find image {src_path}")
            continue
        
        dst_path = os.path.join(output_dir, filename)
        shutil.copy2(src_path, dst_path)

In [14]:
def save_json(data, output_path):
    """Save data to JSON file"""
    with open(output_path, 'w') as f:
        json.dump(data, f, indent=2)

In [15]:
# ==================== DATASET REGISTRATION ====================

def register_page_datasets():
    """Register PAGE-XML derived COCO datasets for Detectron2"""
    
    # Dataset paths
    datasets = {
        "page_train": {
            "json_file": "page_train.json",
            "image_root": "page_train"
        },
        "page_val": {
            "json_file": "page_val.json", 
            "image_root": "page_val"
        },
        "page_test": {
            "json_file": "page_test.json",
            "image_root": "page_test"
        }
    }
    
    # Class names - category_id 0=textline, 1=baseline
    thing_classes = ["textline", "baseline"]
    
    # Clear existing registrations
    for dataset_name in datasets.keys():
        if dataset_name in DatasetCatalog.list():
            DatasetCatalog.remove(dataset_name)
    
    # Register each dataset
    for dataset_name, dataset_info in datasets.items():
        json_file = dataset_info["json_file"]
        image_root = dataset_info["image_root"]
        
        # Check if files exist
        if not os.path.exists(json_file):
            print(f"Warning: {json_file} not found.")
            continue
        if not os.path.exists(image_root):
            print(f"Warning: {image_root} directory not found.")
            continue
        
        # Register dataset
        DatasetCatalog.register(
            dataset_name,
            lambda json_f=json_file, img_root=image_root: load_coco_json(
                json_f, img_root, dataset_name
            )
        )
        
        # Set metadata
        MetadataCatalog.get(dataset_name).set(
            json_file=json_file,
            image_root=image_root,
            evaluator_type="coco",
            thing_classes=thing_classes
        )
    
    print("PAGE datasets registered successfully!")

In [16]:
# ==================== TRAINING SETUP ====================

class PageTrainer(DefaultTrainer):
    """Custom trainer for PAGE dataset"""
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """Build COCO evaluator for validation"""
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, output_dir=output_folder)

In [17]:
def setup_cfg():
    """Setup configuration for training"""
    cfg = get_cfg()
    
    # Load base config
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
    
    # Model configuration
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
    cfg.MODEL.RESNETS.DEPTH = 101
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # textline, baseline
    
    # Dataset configuration
    cfg.DATASETS.TRAIN = ("page_train",)
    cfg.DATASETS.TEST = ("page_val",)
    
    # Solver configuration
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.0001
    cfg.SOLVER.MAX_ITER = 15000
    cfg.SOLVER.STEPS = [10000, 14000]
    cfg.SOLVER.CHECKPOINT_PERIOD = 1000
    
    # Test configuration
    cfg.TEST.EVAL_PERIOD = 1000
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    
    # Dataloader configuration
    cfg.DATALOADER.NUM_WORKERS = 2  # Reduced for Kaggle
    
    # Output directory
    cfg.OUTPUT_DIR = "./output_page"
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    
    return cfg

In [18]:
# ==================== INFERENCE FUNCTIONS ====================

def run_inference_on_sample():
    """Run inference on a test sample and visualize results"""
    
    # Setup configuration for inference
    cfg = setup_cfg()
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/kaggle/input/coco-instancesegmentationmask_rcnn_r_101_fpn_3x/pytorch/default/1/model_final_a3ec72.pkl")
    
    # Create predictor
    predictor = DefaultPredictor(cfg)
    
    # Get a sample image from test set
    test_img_dir = "page_test"
    test_images = [f for f in os.listdir(test_img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not test_images:
        print("No test images found!")
        return
    
    # Use first test image
    sample_image = test_images[0]
    image_path = os.path.join(test_img_dir, sample_image)
    
    print(f"Running inference on: {sample_image}")
    
    # Load image
    im = cv2.imread(image_path)
    
    # Run inference
    outputs = predictor(im)
    
    # Visualize results
    metadata = MetadataCatalog.get("page_test")
    v = Visualizer(im[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Display results
    plt.figure(figsize=(12, 8))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.axis('off')
    plt.title(f'Inference Results: {sample_image}')
    plt.show()
    
    # Print detection summary
    instances = outputs["instances"]
    num_instances = len(instances)
    print(f"\nDetected {num_instances} instances:")
    
    if num_instances > 0:
        scores = instances.scores.cpu().numpy()
        classes = instances.pred_classes.cpu().numpy()
        class_names = metadata.thing_classes
        
        for i in range(num_instances):
            class_name = class_names[classes[i]]
            score = scores[i]
            print(f"  {class_name}: {score:.3f}")

In [19]:
# ==================== MAIN EXECUTION ====================

def main():
    """Main execution function"""
    
    print("=== PAGE-XML Detectron2 Training Pipeline ===\n")
    
    # Step 1: Prepare dataset
    print("Step 1: Preparing dataset...")
    
    # Update these paths according to your Kaggle dataset structure
    train_json_path = '/kaggle/input/read-icdar-2019/read-icdar2019/train.json'
    eval_json_path = '/kaggle/input/read-icdar-2019/read-icdar2019/eval.json'
    
    # Check if input files exist
    if not os.path.exists(train_json_path):
        print(f"Error: {train_json_path} not found!")
        print("Please update the dataset paths in the code.")
        return
    
    if not os.path.exists(eval_json_path):
        print(f"Error: {eval_json_path} not found!")
        print("Please update the dataset paths in the code.")
        return
    
    # Merge datasets
    merged_data = merge_datasets(train_json_path, eval_json_path)
    print(f"Total images: {len(merged_data['images'])}")
    print(f"Total annotations: {len(merged_data['annotations'])}")
    
    # Split dataset
    train_data, val_data, test_data = split_dataset(merged_data, seed=42)
    print(f"Train: {len(train_data['images'])} images, {len(train_data['annotations'])} annotations")
    print(f"Val: {len(val_data['images'])} images, {len(val_data['annotations'])} annotations")
    print(f"Test: {len(test_data['images'])} images, {len(test_data['annotations'])} annotations")
    
    # Save JSON files
    save_json(train_data, "page_train.json")
    save_json(val_data, "page_val.json")
    save_json(test_data, "page_test.json")
    
    # Copy images
    print("Copying images...")
    copy_images(train_data['images'], "page_train")
    copy_images(val_data['images'], "page_val")
    copy_images(test_data['images'], "page_test")
    
    # Step 2: Register datasets
    print("\nStep 2: Registering datasets...")
    register_page_datasets()
    
    # Step 3: Setup and train
    print("\nStep 3: Setting up training...")
    cfg = setup_cfg()
    
    # Print configuration summary
    print(f"Base LR: {cfg.SOLVER.BASE_LR}")
    print(f"Max iterations: {cfg.SOLVER.MAX_ITER}")
    print(f"Batch size: {cfg.SOLVER.IMS_PER_BATCH}")
    print(f"Classes: {MetadataCatalog.get('page_train').thing_classes}")
    
    # Create trainer and start training
    print("\nStep 4: Starting training...")
    trainer = PageTrainer(cfg)
    trainer.resume_or_load(resume=False)
    
    # Train the model
    trainer.train()
    
    print("\nTraining completed!")
    
    # Step 5: Run inference on test sample
    print("\nStep 5: Running inference on test sample...")
    run_inference_on_sample()
    
    print("\n=== Pipeline completed successfully! ===")

In [ ]:
# Run the main pipeline
if __name__ == "__main__":
    main()

=== PAGE-XML Detectron2 Training Pipeline ===

Step 1: Preparing dataset...
Loading datasets...
Fixing annotations (adding missing area fields)...
Total images: 1510
Total annotations: 192877
Train: 1057 images, 131530 annotations
Val: 226 images, 34005 annotations
Test: 227 images, 27342 annotations
Copying images...

Step 2: Registering datasets...
PAGE datasets registered successfully!

Step 3: Setting up training...
Base LR: 0.0001
Max iterations: 15000
Batch size: 2
Classes: ['textline', 'baseline']

Step 4: Starting training...
[07/30 04:44:54 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stri

model_final_a3ec72.pkl: 254MB [00:00, 269MB/s]                               


[07/30 04:44:58 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[07/30 04:45:22 d2.utils.events]:  eta: 3:10:15  iter: 19  total_loss: 9.378  loss_cls: 1.17  loss_box_reg: 0.0333  loss_mask: 0.6939  loss_rpn_cls: 6.092  loss_rpn_loc: 1.309    time: 0.8539  last_time: 0.7289  data_time: 0.2956  last_data_time: 0.0035   lr: 1.9981e-06  max_mem: 10698M


2025-07-30 04:45:27.030497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753850727.371686      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753850727.469521      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[07/30 04:45:58 d2.utils.events]:  eta: 3:07:37  iter: 39  total_loss: 8.242  loss_cls: 1.074  loss_box_reg: 0.02448  loss_mask: 0.6926  loss_rpn_cls: 4.769  loss_rpn_loc: 1.455    time: 0.8250  last_time: 0.5295  data_time: 0.1049  last_data_time: 0.0021   lr: 3.9961e-06  max_mem: 10698M
[07/30 04:46:16 d2.utils.events]:  eta: 3:07:16  iter: 59  total_loss: 6.353  loss_cls: 0.9883  loss_box_reg: 0.03744  loss_mask: 0.6899  loss_rpn_cls: 3.121  loss_rpn_loc: 1.364    time: 0.8406  last_time: 0.7535  data_time: 0.1497  last_data_time: 0.0042   lr: 5.9941e-06  max_mem: 10698M
[07/30 04:46:33 d2.utils.events]:  eta: 3:07:28  iter: 79  total_loss: 4.512  loss_cls: 0.9053  loss_box_reg: 0.04968  loss_mask: 0.6876  loss_rpn_cls: 1.578  loss_rpn_loc: 1.279    time: 0.8402  last_time: 0.5973  data_time: 0.1599  last_data_time: 0.0115   lr: 7.9921e-06  max_mem: 10698M
[07/30 04:46:53 d2.utils.events]:  eta: 3:10:45  iter: 99  total_loss: 3.752  loss_cls: 0.8534  loss_box_reg: 0.1146  loss_mask: